In [2]:
# Import libraries
from googleapiclient.discovery import build
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('API_KEY')
CSE_ID = os.getenv('CSE_ID')

# Fungsi untuk mencari hasil di Google Custom Search
def google_search(query, api_key, cse_id, num_results=10):  # Mengubah jumlah hasil menjadi 10
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id, num=num_results).execute()
    search_results = []  # List untuk menyimpan hasil pencarian dalam format dictionary

    # Menyusun hasil pencarian dalam bentuk dictionary
    for item in res.get('items', []):  # Gunakan get untuk menghindari KeyError
        result = {
            'title': item.get('title', 'No title available'),  # Gunakan get untuk menangani key yang hilang
            'snippet': item.get('snippet', 'No snippet available'),  # Periksa dan beri nilai default
            'link': item.get('link', 'No link available')  # Periksa dan beri nilai default
        }
        search_results.append(result)
      
    return search_results  # Mengembalikan daftar hasil pencarian berupa dictionary

# Fungsi untuk menghitung cosine similarity
def calculate_cosine_similarity(query, search_results):
    # Gabungkan query dengan hasil pencarian
    all_texts = [query] + [result['title'] + " " + result['snippet'] for result in search_results]

    # Vectorisasi teks dengan TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_texts)

    # Hitung cosine similarity antara query dan hasil pencarian
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return cosine_sim[0]  # Mengembalikan array 1D yang berisi similarity

# Contoh judul tugas dan pengumpulan hasil pencarian
judul_tugas = "Limit,turunan,dan integral"
hasil_pencarian = google_search(judul_tugas, api_key, cse_id, num_results=10)

# Menampilkan hasil pencarian
print("Hasil Pencarian untuk:", judul_tugas)
for idx, result in enumerate(hasil_pencarian, start=1):
    print(f"{idx}. Title: {result['title']}")
    print(f"   Snippet: {result['snippet']}")
    print(f"   Link: {result['link']}\n")  # Menampilkan link

# Menghitung similarity antara judul tugas dan hasil pencarian
cosine_sim = calculate_cosine_similarity(judul_tugas, hasil_pencarian)

# Menampilkan hasil similarity
print("\nCosine Similarity antara judul tugas dan hasil pencarian:")
for idx, sim in enumerate(cosine_sim, start=1):
    print(f"Similarity dengan hasil pencarian {idx}: {sim:.4f}")

# Mengurutkan hasil berdasarkan similarity tertinggi
sorted_sim_indices = sorted(range(len(cosine_sim)), key=lambda i: cosine_sim[i], reverse=True)

# Menampilkan 3 hasil dengan similarity tertinggi
print("\nTop 3 Cosine Similarity antara judul tugas dan hasil pencarian:")
for idx in sorted_sim_indices[:3]:
    print(f"Similarity dengan hasil pencarian {idx+1}: {cosine_sim[idx]:.4f}")
    print(f"   Title: {hasil_pencarian[idx]['title']}")
    print(f"   Snippet: {hasil_pencarian[idx]['snippet']}")
    print(f"   Link: {hasil_pencarian[idx]['link']}\n")


Hasil Pencarian untuk: Limit,turunan,dan integral
1. Title: Kupas Tuntas Rumus Kalkulus Dasar: Limit, Turunan, dan Integral
   Snippet: Sep 20, 2021 ... Integral (Kalkulus Dasar). kalkulus dasar. Integral merupakan kebalikan dari definisi turunan. Jika suatu fungsi diturunkan terhadap x, maka ...
   Link: https://www.zenius.net/blog/rumus-kalkulus-dasar-limit-turunan-dan-integral

2. Title: Pengertian dan Rumus Kalkulus Dasar: Limit, Turunan, dan Integral ...
   Snippet: Jun 14, 2022 ... Artinya, jika f(x) mendekati suatu nilai L ketika x mendekati c dari arah kiri ataupun kanan maka limit f(x) dengan x mendekati c adalah L.
   Link: https://kumparan.com/kabar-harian/pengertian-dan-rumus-kalkulus-dasar-limit-turunan-dan-integral-1yGlrh9V8p7

3. Title: Limit, Turunan, Dan Integral | PDF
   Snippet: Informasi Dokumen ... Modul ini membahas limit, turunan, dan integral dalam MATLAB. Terdapat berbagai jenis limit seperti limit kiri, kanan, dan trigonometri yang ...
   Link: https://id.scri